In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import Libraries

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.cm as cm
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import accuracy_score
from keras.models import Model
from keras import layers, losses


from keras.models import Sequential
from keras.layers import InputLayer, LSTM, Dense, RepeatVector, TimeDistributed
from keras.utils import plot_model

### Load data

In [5]:
DATA_DIR_NORMAL = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/data/anomalyDetection_Normal.npy"
DATA_DIR_ISSUE = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/data/anomalyDetection_Issue.npy"

# Set style for matplotlib
plt.style.use("Solarize_Light2")

In [6]:
data_normal = np.load(DATA_DIR_NORMAL)
data_issue = np.load(DATA_DIR_ISSUE)

data_normal = data_normal.reshape((data_normal.shape[0], data_normal.shape[1], 1))
data_issue = data_issue.reshape((data_issue.shape[0], data_issue.shape[1], 1))

## Model

In [8]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [52]:
input_shape = (data_normal.shape[1:])
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=input_shape, kernel_initializer="random_normal", bias_initializer='zeros'))
model.add(RepeatVector(input_shape[0]))
model.add(LSTM(200, activation='relu', return_sequences=True, kernel_initializer="random_normal", bias_initializer='zeros'))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 200)               161600    
                                                                 
 repeat_vector_4 (RepeatVect  (None, 85, 200)          0         
 or)                                                             
                                                                 
 lstm_9 (LSTM)               (None, 85, 200)           320800    
                                                                 
 time_distributed_4 (TimeDis  (None, 85, 1)            201       
 tributed)                                                       
                                                                 
Total params: 482,601
Trainable params: 482,601
Non-trainable params: 0
_________________________________________________________________


In [29]:
# should not be trained
model.fit(data_normal, data_normal, epochs=20)

Epoch 1/20
32/32 [==============================] - 6s 151ms/step - loss: 0.2819
Epoch 2/20
32/32 [==============================] - 5s 166ms/step - loss: 0.2812
Epoch 3/20
32/32 [==============================] - 6s 172ms/step - loss: 0.2828
Epoch 4/20
32/32 [==============================] - 6s 189ms/step - loss: 0.2809
Epoch 5/20
32/32 [==============================] - 6s 190ms/step - loss: 54.9586
Epoch 6/20
32/32 [==============================] - 6s 187ms/step - loss: 0.2840
Epoch 7/20
32/32 [==============================] - 6s 190ms/step - loss: 0.2808
Epoch 8/20
32/32 [==============================] - 6s 185ms/step - loss: 0.2798
Epoch 9/20
32/32 [==============================] - 6s 184ms/step - loss: 0.2625
Epoch 10/20
32/32 [==============================] - 6s 182ms/step - loss: 0.2303
Epoch 11/20
32/32 [==============================] - 6s 183ms/step - loss: 0.2208
Epoch 12/20
32/32 [==============================] - 6s 184ms/step - loss: 0.2172
Epoch 13/20
32/32 [=====

In [53]:
y_train = model.predict(data_normal)

cost_train = y_train - data_normal
print(np.round(cost_train,3), f"mean: {np.round(cost_train.mean(),3)}")

32/32 [==============================] - 2s 44ms/step
[[[ 0.871]
  [ 0.452]
  [ 0.007]
  ...
  [ 0.888]
  [-0.218]
  [ 1.007]]

 [[ 0.871]
  [ 0.452]
  [ 0.007]
  ...
  [ 0.888]
  [-0.218]
  [ 1.007]]

 [[ 0.871]
  [ 0.452]
  [ 0.007]
  ...
  [ 0.888]
  [-0.218]
  [ 1.007]]

 ...

 [[ 0.618]
  [ 0.657]
  [ 0.507]
  ...
  [ 0.989]
  [ 0.204]
  [ 1.008]]

 [[ 0.618]
  [ 0.657]
  [ 0.507]
  ...
  [ 0.995]
  [ 0.275]
  [ 1.008]]

 [[ 0.618]
  [ 0.657]
  [ 0.507]
  ...
  [ 0.997]
  [ 0.307]
  [ 1.008]]] mean: 0.09000000357627869


In [54]:
y_test = model.predict(data_issue)

cost_test = y_test - data_issue
print(np.round(cost_test,3), f"mean: {cost_test.mean()}")

13/13 [==============================] - 1s 42ms/step
[[[-0.169]
  [-0.054]
  [ 0.52 ]
  ...
  [-0.744]
  [-0.665]
  [ 1.369]]

 [[-0.169]
  [-0.054]
  [ 0.52 ]
  ...
  [-0.746]
  [-0.663]
  [ 1.369]]

 [[ 0.256]
  [ 0.135]
  [ 0.52 ]
  ...
  [-0.746]
  [-0.664]
  [ 1.369]]

 ...

 [[ 0.576]
  [ 0.383]
  [ 0.52 ]
  ...
  [-0.454]
  [-0.882]
  [ 1.369]]

 [[ 0.576]
  [ 0.383]
  [ 0.52 ]
  ...
  [-0.41 ]
  [-0.902]
  [ 1.369]]

 [[ 0.576]
  [ 0.383]
  [ 0.52 ]
  ...
  [-0.375]
  [-0.917]
  [ 1.369]]] mean: 0.0955331102013588
